In [1]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
import time

import numpy as np
from multiprocess import Pool

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *

%matplotlib inline

Setting environment for Precision WorkStation


No vtk
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
atlas_spec = dict(name='atlasV6',
                   vol_type='score'    ,               
                    resolution='10.0um'
                   )

In [3]:
atlas_structures_wrt_canonicalAtlasSpace_atlasResol = \
DataManager.load_original_volume_all_known_structures_v3(atlas_spec, in_bbox_wrt='canonicalAtlasSpace',
                                                        out_bbox_wrt='canonicalAtlasSpace')

Prior structure/index map not found. Generating a new one.


In [4]:
stack = 'CHATM3'

In [5]:
T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol = bp.unpack_ndarray_file('/home/yuncong/' + stack + '_T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol.bp')

In [6]:
registered_atlas_structures_wrt_wholebrainWithMargin_atlasResol = \
{name_s: transform_volume_v4(volume=vo, transform=T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol, return_origin_instead_of_bbox=True)
for name_s, vo in atlas_structures_wrt_canonicalAtlasSpace_atlasResol.iteritems()}

/home/yuncong/Brain/utilities/registration_utilities.py:1561: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(volume_m_aligned_to_f.dtype, np.float):
Interpolating/filling sparse volume: 0.07 seconds.
Interpolating/filling sparse volume: 0.33 seconds.
Interpolating/filling sparse volume: 0.09 seconds.
Interpolating/filling sparse volume: 0.06 seconds.
Interpolating/filling sparse volume: 0.02 seconds.
Interpolating/filling sparse volume: 0.14 seconds.
Interpolating/filling sparse volume: 0.14 seconds.
Interpolating/filling sparse volume: 0.06 seconds.
Interpolating/filling sparse volume: 0.02 seconds.
Interpolating/filling sparse volume: 0.21 seconds.
Interpolating/filling sparse volume: 0.33 seconds.
Interpolating/filling sparse volume: 0.09 seconds.
Interpolating/filling sparse volume: 0.07 seconds.
Interpolating/filling sparse volume: 

In [7]:
# Define volume ROI for which to compute scores based on outline registered atlas.

In [8]:
registered_atlas_structures_bbox_wrt_wholebrainWithMargin_atlasResol = \
{name_s: (o[0], o[0] + v.shape[1] - 1, o[1], o[1] + v.shape[0] - 1, o[2], o[2] + v.shape[2] - 1)
 for name_s, (v, o) in registered_atlas_structures_wrt_wholebrainWithMargin_atlasResol.iteritems()}

In [9]:
registered_atlas_structures_xyzTwoCorners_wrt_wholebrainWithMargin_atlasResol = \
{name_s: ((o[0], o[2], o[4]), (o[1], o[3], o[5]))
for name_s, o in registered_atlas_structures_bbox_wrt_wholebrainWithMargin_atlasResol.iteritems()}

In [10]:
registered_atlas_structures_xyzCorners_wrt_wholebrainWithMargin_atlasResol = \
{name_s: ((o[0], o[2], o[4]), (o[0], o[2], o[5]), (o[0], o[3], o[4]), (o[0], o[3], o[5]), \
         (o[1], o[2], o[4]), (o[1], o[2], o[5]), (o[1], o[3], o[4]), (o[1], o[3], o[5]))
for name_s, o in registered_atlas_structures_bbox_wrt_wholebrainWithMargin_atlasResol.iteritems()}

In [11]:
from data_manager import CoordinatesConverter

In [12]:
converter = CoordinatesConverter(stack=stack)

In [13]:
# registered_atlas_structures_wrt_wholebrainXYcropped_xysec = {}

# for name_s, corners_xyz in registered_atlas_structures_xyzCorners_wrt_wholebrainWithMargin_atlasResol.iteritems():
# #     print name_s
#     registered_atlas_structures_wrt_wholebrainXYcropped_xysec[name_s] = \
#     converter.convert_frame_and_resolution(p=corners_xyz, 
#                                        in_wrt=('wholebrainWithMargin', 'sagittal'),
#                                       in_resolution='10.0um',
#                                       out_wrt=('wholebrainXYcropped', 'sagittal'),
#                                       out_resolution='image_image_section').astype(np.int)

In [14]:
registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners = {}

for name_s, corners_xyz in registered_atlas_structures_xyzTwoCorners_wrt_wholebrainWithMargin_atlasResol.iteritems():
#     print name_s
    registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners[name_s] = \
    converter.convert_frame_and_resolution(p=corners_xyz, 
                                       in_wrt=('wholebrainWithMargin', 'sagittal'),
                                      in_resolution='10.0um',
                                      out_wrt=('wholebrainXYcropped', 'sagittal'),
                                      out_resolution='image_image_section').astype(np.int)

In [15]:
registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners

{'10N_L': array([[15084, 12156,   203],
        [19422, 14525,   229]]), '10N_R': array([[15084, 12156,   232],
        [19422, 14525,   258]]), '12N': array([[14961, 12525,   209],
        [19053, 14710,   251]]), '3N_L': array([[ 4715,  9264,   216],
        [ 6776, 10525,   230]]), '3N_R': array([[ 4715,  9264,   231],
        [ 6776, 10525,   245]]), '4N_L': array([[ 6591,  9510,   213],
        [ 7238, 10279,   223]]), '4N_R': array([[ 6591,  9510,   238],
        [ 7238, 10279,   248]]), '5N_L': array([[ 7884, 11356,   147],
        [10376, 14217,   176]]), '5N_R': array([[ 7884, 11356,   285],
        [10376, 14217,   314]]), '6N_L': array([[10561, 12340,   206],
        [11453, 13202,   218]]), '6N_R': array([[10561, 12340,   243],
        [11453, 13202,   255]]), '7N_L': array([[10407, 14648,   149],
        [13638, 17356,   196]]), '7N_R': array([[10407, 14648,   265],
        [13638, 17356,   312]]), '7n_L': array([[ 8991, 12094,   141],
        [12038, 15848,   216]]), '7n_

In [16]:
save_json(registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners, 
          '/home/yuncong/' + stack + '_registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners.json')

In [15]:
# registered_atlas_structures_wrt_wholebrainWithMargin_section_limits = {}

# for name_s, b in registered_atlas_structures_bbox_wrt_wholebrainWithMargin_atlasResol.iteritems():
# #     print name_s
#     registered_atlas_structures_wrt_wholebrainWithMargin_section_limits[name_s] = \
#     converter.convert_frame_and_resolution(p=[[np.nan, np.nan, b[4]], [np.nan, np.nan, b[5]]], 
#                                        in_wrt=('wholebrainWithMargin', 'sagittal'),
#                                       in_resolution='10.0um',
#                                       out_wrt='wholebrain',
#                                       out_resolution='image_image_section')[:,2].astype(np.int)

In [35]:
# registered_atlas_structures_wrt_wholebrainWithMargin_section_limits

In [36]:
# save_json(registered_atlas_structures_wrt_wholebrainWithMargin_section_limits, 
#           '/home/yuncong/' + stack + '_structure_section_limits.json')

In [4]:
batch_size = 256
model_dir_name = 'inception-bn-blue'
model_name = 'inception-bn-blue'
model, mean_img = load_mxnet_model(model_dir_name=model_dir_name, model_name=model_name, 
                                   num_gpus=1, batch_size=batch_size)

/usr/local/lib/python2.7/dist-packages/mxnet/module/base_module.py:53: UserWarning: You created Module with Module(..., label_names=['softmax_label']) but input with name 'softmax_label' is not found in symbol.list_arguments(). Did you mean one of:
	data
  warnings.warn(msg)
/usr/local/lib/python2.7/dist-packages/mxnet/module/base_module.py:65: UserWarning: Data provided by label_shapes don't match names specified by label_names ([] vs. ['softmax_label'])
  warnings.warn(msg)


In [7]:
# out_resolution_um = 10.
# out_resolution_str = '%.1fum' % out_resolution_um

# detector_id = 998
detector_id = 799
# print 'input_version:', detector_settings.loc[detector_id]['input_version']

In [8]:
detector_setting = {}
detector_setting['feature_classifier_id'] = 899
detector_setting['windowing_id'] = 7

In [9]:
clfs = DataManager.load_classifiers(classifier_id=detector_setting['feature_classifier_id'])

In [10]:
win_id = detector_setting['windowing_id']

In [11]:
motor_nuclei = ['Amb', '3N', '4N', '5N', '6N', '7N', '10N', '12N']

In [12]:
# output_resolution = '10.0um'

In [ ]:
for stack in ['CHATM2', 'CHATM3']:
    
    output_resolution = '10.0um'
    out_resolution_um = convert_resolution_string_to_um(resolution=output_resolution, stack=stack)
    
    registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners = \
load_json('/home/yuncong/' + stack + '_registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners.json')

    ####################

    registered_atlas_structures_wrt_wholebrainXYcropped_bboxes_perSection = defaultdict(dict)

    section_margin_um = 500.
    section_margin = int(section_margin_um / SECTION_THICKNESS)
    image_margin_um = 500.
    image_margin = int(np.round(image_margin_um / convert_resolution_string_to_um('raw', stack)))

    valid_secmin = np.min(metadata_cache['valid_sections'][stack])
    valid_secmax = np.max(metadata_cache['valid_sections'][stack])

    # for name_u in all_known_structures:
    for name_u in motor_nuclei:
    # for name_u in ['RtTg', 'VCA', 'AP', 'Tz', 'LC', 'Sp5O', '7n', 'SNC', 'Sp5I', 'DC', 'Sp5C', 'VCP', 'RMC', 'PBG', 'SNR', 'LRt', 'SC', 'IC', 'Pn', 'VLL']:

        if name_u in singular_structures:

            (xmin, ymin, secmin), (xmax, ymax, secmax) = registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners[name_u]

            for sec in range(max(secmin - section_margin, valid_secmin), min(secmax + 1 + section_margin, valid_secmax)):

                if is_invalid(sec=sec, stack=stack):
                    continue

                registered_atlas_structures_wrt_wholebrainXYcropped_bboxes_perSection[name_u][sec] = \
                (max(xmin - image_margin, 0), 
                 xmax + image_margin, 
                 max(ymin - image_margin, 0), 
                 ymax + image_margin)
        else:

            a = defaultdict(list)

            lname = convert_to_left_name(name_u)        
            (xmin, ymin, secmin), (xmax, ymax, secmax) = registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners[lname]

            for sec in range(max(secmin - section_margin, valid_secmin), min(secmax + 1 + section_margin, valid_secmax)):

                if is_invalid(sec=sec, stack=stack):
                    continue

                a[sec].append((max(xmin - image_margin, 0), 
                 xmax + image_margin, 
                 max(ymin - image_margin, 0), 
                 ymax + image_margin))

            rname = convert_to_right_name(name_u)
            (xmin, ymin, secmin), (xmax, ymax, secmax) = registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners[rname]

            for sec in range(max(secmin - section_margin, valid_secmin), min(secmax + 1 + section_margin, valid_secmax)):

                if is_invalid(sec=sec, stack=stack):
                    continue

                a[sec].append((max(xmin - image_margin, 0), 
                 xmax + image_margin, 
                 max(ymin - image_margin, 0), 
                 ymax + image_margin))

            for sec, bboxes in a.iteritems():
                if len(bboxes) == 1:
                    registered_atlas_structures_wrt_wholebrainXYcropped_bboxes_perSection[name_u][sec] = bboxes[0]
                else:
                    xmin, ymin = np.min(bboxes, axis=0)[[0,2]]
                    xmax, ymax = np.max(bboxes, axis=0)[[1,3]]
                    registered_atlas_structures_wrt_wholebrainXYcropped_bboxes_perSection[name_u][sec] = (xmin, xmax, ymin, ymax)
                    
                    
    ######### Generate score maps ###########
                    
#     for name_u in all_known_structures:
    for name_u in motor_nuclei:
# for name_u in ['10N']:
# for name_u in ['RtTg', 'VCA', 'AP', 'Tz', 'LC', 'Sp5O', '7n', 'SNC', 'Sp5I', 'DC', 'Sp5C', 'VCP', 'RMC', 'PBG', 'SNR', 'LRt', 'SC', 'IC', 'Pn', 'VLL']:
            
        for sec, bbox in sorted(registered_atlas_structures_wrt_wholebrainXYcropped_bboxes_perSection[name_u].items()):

    #         if is_invalid(sec=sec, stack=stack):
    #             continue

            print name_u, sec

            viz_all_landmarks, scoremap_all_landmarks = \
            draw_scoremap(clfs={name_u: clfs[name_u]}, 
                                  bbox=bbox,
                            scheme='none', 
                        win_id=win_id, prep_id=2,
                        stack=stack, return_scoremap=True, sec=sec,
                        model=model, model_name=model_name,
                         mean_img=mean_img, 
                         batch_size=batch_size,
                          output_patch_size=224,
                          is_nissl=False,
                       out_resolution_um=out_resolution_um,
                    image_shape=metadata_cache['image_shape'][stack],
                                  return_wholeimage=True)

            sm = scoremap_all_landmarks[name_u]
            viz = viz_all_landmarks[name_u]

            scoremap_bp_filepath = \
            DataManager.get_downscaled_scoremap_filepath(stack=stack, section=sec, 
                                                         structure=name_u,
                                                         detector_id=detector_id,
                                                         out_resolution_um=out_resolution_um)
            save_data(sm.astype(np.float16), scoremap_bp_filepath, upload_s3=False)

            viz_filepath = \
            DataManager.get_scoremap_viz_filepath_v2(stack=stack, section=sec, 
                                                         structure=name_u,
                                                         detector_id=detector_id,
                                                         out_resolution=output_resolution)
            save_data(viz, viz_filepath, upload_s3=False)

            del viz_all_landmarks, scoremap_all_landmarks
        
    ######### Generate score volumes ##########
        
    # for structure in ['4N']:
    for structure in motor_nuclei:

        for name_s in [convert_to_left_name(structure), convert_to_right_name(structure)]:

            scoremaps = {}

    #         for sec in metadata_cache['valid_sections'][stack]:

            (xmin, ymin, s1), (xmax, ymax, s2) = registered_atlas_structures_wrt_wholebrainXYcropped_xysecTwoCorners[name_s]

            for sec in range(max(s1 - section_margin, metadata_cache['section_limits'][stack][0]), 
                             min(s2 + 1 + section_margin, metadata_cache['section_limits'][stack][1])):

                if is_invalid(sec=sec, stack=stack):
                    continue

                scoremap = DataManager.load_downscaled_scoremap(stack=stack, section=sec, structure=structure, 
                                                                prep_id='alignedBrainstemCrop',
                                                              out_resolution_um=out_resolution_um, 
                                                                detector_id=detector_id).astype(np.float32)

                mask = DataManager.load_image_v2(stack=stack, section=sec, 
                                     prep_id='alignedBrainstemCrop', 
                                     resol='thumbnail', version='mask')

                mask_outResol = rescale_by_resampling(mask, new_shape=(scoremap.shape[1], scoremap.shape[0]))

                scoremap[~mask_outResol] = 0
                scoremaps[sec] = scoremap

            volume_outVolResol, volume_origin_wrt_wholebrainXYcropped_outVolResol = \
            images_to_volume_v2(images=scoremaps, spacing_um=20.,
                                    in_resol_um=out_resolution_um,
                                    out_resol_um=out_resolution_um)
            print volume_outVolResol.shape

            brain_spec = dict(name=stack,
                           vol_type='score',
                            detector_id=detector_id,
                           resolution=output_resolution)

            # Save volume and origin.

            save_data(volume_outVolResol.astype(np.float16), \
                      DataManager.get_original_volume_filepath_v2(stack_spec=brain_spec, structure=name_s))

            wholebrainXYcropped_origin_wrt_wholebrain_outVolResol = \
            DataManager.get_domain_origin(stack=stack, domain='wholebrainXYcropped', 
                                          resolution=output_resolution)
            volume_origin_wrt_wholebrain_outVolResol =\
            volume_origin_wrt_wholebrainXYcropped_outVolResol + wholebrainXYcropped_origin_wrt_wholebrain_outVolResol

            save_data(volume_origin_wrt_wholebrain_outVolResol,
                      DataManager.get_original_volume_origin_filepath_v3(stack_spec=brain_spec, structure=name_s, wrt='wholebrain'))

            # Compute gradients.

            gradients = compute_gradient_v2((volume_outVolResol, volume_origin_wrt_wholebrain_outVolResol), 
                                            smooth_first=True)

            DataManager.save_volume_gradients(gradients, stack_spec=brain_spec, structure=name_s)

Amb 89


locate patches: 0.05 seconds
Load pre-computed features: 0.87 seconds
Not using image_cache.


/data/CSHL_data_processed/CHATM2/CHATM2_prep2_raw_NtbNormalizedAdaptiveInvertedGammaJpeg/CHATM2_slide29_2018_01_18-S3_prep2_raw_NtbNormalizedAdaptiveInvertedGammaJpeg.jpg


Load background image: 2.33 seconds
/usr/local/lib/python2.7/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
Rescale background image to output resolution: 0.21 seconds
Predict scores Amb: 0.01 seconds
Resample scoremap Amb: 0.09 seconds
Load and rescale background image: 0.00 seconds
/usr/local/lib/python2.7/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
Generate scoremap overlay: 0.06 seconds.
Scoremap size does not match background image size. Need to resize: 0.08 seconds.
Generate scoremap overlay image Amb: 0.17 seconds
locate patches: 0.05 seconds


(597, 925, 3) (597, 925)
Amb 90


...

Load background image: 2.24 seconds
Rescale background image to output resolution: 0.34 seconds
Predict scores 12N: 0.01 seconds
Resample scoremap 12N: 0.09 seconds
Load and rescale background image: 0.00 seconds
Generate scoremap overlay: 0.06 seconds.
Scoremap size does not match background image size. Need to resize: 0.08 seconds.
Generate scoremap overlay image 12N: 0.16 seconds
locate patches: 0.02 seconds


(597, 925, 3) (597, 925)
12N 155


Load pre-computed features: 0.67 seconds
Not using image_cache.


/data/CSHL_data_processed/CHATM2/CHATM2_prep2_raw_NtbNormalizedAdaptiveInvertedGammaJpeg/CHATM2_slide51_2018_01_22-S3_prep2_raw_NtbNormalizedAdaptiveInvertedGammaJpeg.jpg


Load background image: 2.68 seconds
Rescale background image to output resolution: 0.34 seconds
